#### Custom Collection Implementations

You can use your own types for collections as well. In simple cases, _inheriting_ from `list` or `set`, _adding custom behavior_, is all that's needed. In other cases, _special decorators_ are needed to tell SQLAlchemy __more detail about how the collection operates__.

> ###### Do I need a custom collection implementation?
>
> In most cases not at all! The most common use cases for a "custom" collection is one that _validates or marshals incoming values into a new form_, such as a string that becomes a class instance, or one which goes a step beyond and _represents the data internally in some fashion_, presenting a "view" of that data on the outside of a different form.
>
> For the first use case, the `validates()` decorator is by far the _simplest way to intercept incoming values_ in all cases for the purposes of __validation and simple marshaling__.
>
> For the second use case, the _Association Proxy extension_ is a _well-tested_, _widely used_ system that provides a __read/write "view" of a collection__ in terms of some attribute present on the target object. As the _target attribute_ can be a `@property` that returns virtually _anything_, a _wide array of "alternative" views_ of a collection can be constructed with just a few functions. This approach leaves the __underlying mapped collection unaffected and avoids the need to carefully tailor collection behavior on a method-by-method basis__.
>
> _Customized collections_ are useful when the collection needs to have _special behaviors upon access or mutation operations_ that __can't otherwise be modeled externally__ to the collection. They can of course be __combined__ with the _above two approaches_.

Collections in SQLAlchemy are _transparently instrumented_. `Instrumentation` means that _normal operations on the collection are tracked and result in changes being written to the database at flush time_. Additionally, collection operations __can fire events__ which indicate some secondary operation must take place. Examples of a secondary operation include saving the child item in the parent's _Session_ (i.e. the _save-update cascade_), as well as _synchronizing_ the state of a `bi-directional relationship` (i.e. a `backref()`).

The collections package understands the _basic interface of lists, sets and dicts_ and will __automatically apply instrumentation__ to those built-in types and their subclasses. _Object-derived types_ that implement a basic collection interface are __detected and instrumented via duck-typing__.

In [6]:
from sqlalchemy.util import OrderedDict
from sqlalchemy.orm.collections import collection, MappedCollection

In [1]:
class ListLike(object):
    def __init__(self):
        self.data = []
    
    def append(self, item):
        self.data.append(item)
    
    def remove(self, item):
        self.data.remove(item)
    
    def extend(self, items):
        self.data.extend(items)
    
    def __iter__(self):
        return iter(self.data)
    
    def foo(self):
        return "foo"

`append`, `remove`, and `extend` are known _list-like_ methods, and will be __instrumented automatically__. `__iter__` is __not__ a _mutator method_ and _won't be instrumented_, and foo won't be either.

_Duck-typing_ (i.e. guesswork) __isn't rock-solid__, of course, so you can be _explicit_ about the interface you are implementing by providing an `__emulates__` class attribute.

In [2]:
class SetLike(object):
    __emulates__ = set
    
    def __init__(self):
        self.data = set()
    
    def append(self, item):
        self.data.add(item)
    
    def remove(self, item):
        self.data.remove(item)
    
    def __iter__(self):
        return iter(self.data)

This class looks _list-like_ because of `append`, but `__emulates__` __forces it to set-like__. `remove` is known to be part of the _set interface_ and __will be instrumented__.

But this class __won't work__ quite yet: a __little glue is needed to adapt it__ for use by _SQLAlchemy_. The ORM _needs to know_ which methods to use to _append_, _remove_ and _iterate_ over members of the collection. When using a type like _list_ or _set_, the __appropriate methods are well-known and used automatically__ when present. This _set-like_ class __does not provide the expected add method__, so we __must supply an explicit mapping__ for the ORM _via a decorator_.

##### Annotating Custom Collections via Decorators

_Decorators_ can be used to __tag the individual methods__ the ORM needs to _manage collections_. Use them when your class __doesn't quite meet the regular interface__ for its _container type_, or when you otherwise would like to use a _different method to get the job done_.

In [4]:
class SetLike(object):
    __emulates__ = set
    
    def __init__(self):
        self.data = set()
    
    @collection.appender
    def append(self, item):
        self.data.add(item)
    
    def remove(self, item):
        self.data.remove(item)
    
    def __iter__(self):
        return iter(self.data)

And that's all that's needed to complete the example. SQLAlchemy will _add instances via the append method_. `remove` and `__iter__` are the _default methods_ for sets and will be used for _removing_ and _iteration_. __Default methods can be changed as well__.

In [5]:
class MyList(list):
    @collection.remover
    def zark(self, item):
        # do something special...
        pass
    
    @collection.iterator
    def hey_use_this_instead_for_iteration(self):
        pass

There is __no requirement__ to be _list-_, or _set-like_ at all. __Collection classes can be any shape__, so long as they have the `append`, `remove` and `iterate` interface marked for SQLAlchemy's use. _Append_ and _remove_ methods will be __called with a mapped entity as the single argument__, and _iterator_ methods are called with __no arguments__ and __must return an iterator__.

##### Custom Dictionary-Based Collections

The _MappedCollection_ class can be used as a _base class for your custom types_ or as a _mix-in_ to __quickly add dict collection support to other classes__. It uses a _keying function_ to delegate to `__setitem__` and `__delitem__`.

In [8]:
class NodeMap(MappedCollection):
    """Holds 'Node' objects, keyed by the 'name' attribute with insert order maintained."""
    
    def __init__(self, *args, **kw):
        MappedCollection.__init__(self, keyfunc=lambda node: node.name)
        OrderedDict.__init__(self, *args, **kw)

When subclassing _MappedCollection_, _user-defined_ versions of `__setitem__()` or `__delitem__()` should be __decorated__ with `collection.internally_instrumented()`, if they __call down to those same methods__ on _MappedCollection_. This because the methods on _MappedCollection_ are __already instrumented__ - calling them from within an _already instrumented_ call can __cause events to be fired off repeatedly__, or __inappropriately__, leading to `internal state corruption` in rare cases.

In [9]:
class MyMappedCollection(MappedCollection):
    """Use @internally_instrumented when your methods
    call down to already-instrumented methods.
    """
    
    @collection.internally_instrumented
    def __setitem__(self, key, value, _sa_initiator=None):
        # do something with key, value
        super(MyMappedCollection, self).__setitem__(key, value, _sa_initiator)
    
    @collection.internally_instrumented
    def __delitem__(self, key, _sa_initiator=None):
        # do something with key
        super(MyMappedCollection, self).__delitem__(key, _sa_initiator)

The ORM understands the _dict interface_ just like _lists_ and _sets_, and will __automatically instrument all dict-like methods__ if you choose to _subclass dict or provide dict-like collection behavior_ in a duck-typed class. You __must decorate appender and remover methods__, however- there are _no compatible methods in the basic dictionary interface_ for SQLAlchemy to use by default. Iteration will go through `itervalues()` _unless otherwise decorated_.

##### Instrumentation and Custom Types

Many _custom types_ and _existing library classes_ can be used as a `entity collection type` __as-is without further ado__. However, it is important to note that the _instrumentation_ process will _modify the type_, _adding decorators_ around methods __automatically__.

The _decorations_ are __lightweight and no-op outside of relationships__, but they do __add unneeded overhead when triggered elsewhere__. When using a library class as a collection, it can be good practice to use the _"trivial subclass"_ trick to _restrict the decorations_ to just your usage in relationships.

```
class MyAwesomeList(some.great.library.AwesomeList):
    pass


# ... relationship(..., collection_class=MyAwesomeList)
```

The ORM uses this approach for _built-ins_, __quietly substituting a trivial subclass__ when a `list`, `set` or `dict` is _used directly_.